In [1]:
import pandas as pd
import datetime as dt

In [2]:
dataset=pd.read_csv("Chiller con week days.csv")

In [3]:
dataset

,DAYS,DATE,TOTAL A,TOTAL B,TOTAL C
0,Monday,2023-10-09,8.96,7.84,5.80
1,Tuesday,2023-10-10,10.82,9.87,5.37
2,Wednesday,2023-10-11,9.46,9.46,6.07
3,Thursday,2023-10-12,9.26,11.12,4.38
4,Friday,2023-10-13,8.27,11.00,3.94
...,...,...,...,...,...
84,Wednesday,2024-02-07,7.81,8.62,2.86
85,Thursday,2024-02-08,5.37,4.56,2.97
86,Friday,2024-02-09,6.86,7.89,2.67
87,Monday,2024-02-12,3.02,1.49,1.56


In [4]:
dataset.columns

Index(['DAYS', 'DATE', 'TOTAL A', 'TOTAL B', 'TOTAL C'], dtype='object')

In [5]:
dataset.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   DAYS     89 non-null     object 
 1   DATE     89 non-null     object 
 2   TOTAL A  89 non-null     float64
 3   TOTAL B  89 non-null     float64
 4   TOTAL C  89 non-null     float64
dtypes: float64(3), object(2)
memory usage: 3.6+ KB


In [6]:
dataset['DATE']=pd.to_datetime(dataset['DATE'])

In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   DAYS     89 non-null     object        
 1   DATE     89 non-null     datetime64[ns]
 2   TOTAL A  89 non-null     float64       
 3   TOTAL B  89 non-null     float64       
 4   TOTAL C  89 non-null     float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 3.6+ KB


In [8]:
startdate="2023-10-09"
enddate="2024-02-13"
df=dataset[dataset['DATE'].between(startdate,enddate)][['DATE', 'TOTAL A']]

In [9]:
df

,DATE,TOTAL A
0,2023-10-09,8.96
1,2023-10-10,10.82
2,2023-10-11,9.46
3,2023-10-12,9.26
4,2023-10-13,8.27
...,...,...
84,2024-02-07,7.81
85,2024-02-08,5.37
86,2024-02-09,6.86
87,2024-02-12,3.02


In [10]:
column='TOTAL A'

In [11]:
from sklearn.preprocessing import MinMaxScaler
Ms=MinMaxScaler()
data1=Ms.fit_transform(df[[column]])
print("Len:",data1.shape)

Len: (89, 1)


In [12]:
len(data1)

89

In [13]:
training_size=round(len(data1)*0.80)
print(training_size)
X_train=data1[:training_size]
X_test=data1[training_size:]
print("X_train length:",X_train.shape)
print("X_test length:",X_test.shape)
y_train=data1[:training_size]
y_test=data1[training_size:]
print("y_train length:",y_train.shape)
print("y_test length:",y_test.shape)

71
X_train length: (71, 1)
X_test length: (18, 1)
y_train length: (71, 1)
y_test length: (18, 1)


In [14]:
import warnings
warnings.filterwarnings("ignore")

In [15]:
#pip install statsmodels

In [16]:
from sklearn.metrics import mean_squared_error
trends=['n','t','c','ct']
orders=[(0,0,1),(0,0,2),(0,0,3)]
from statsmodels.tsa.arima.model import ARIMA
for td in trends:
        
        #print(td)
        #trendslist.append(td)
    model = ARIMA(X_train, order=(0,0,10),trend=td,)
    model_fit = model.fit()
        # make prediction
    y_pred= model_fit.predict(len(X_train), len(data1)-1)
        #print(y_pred)
    from sklearn.metrics import r2_score
    mse=mean_squared_error(y_test,y_pred,squared=False)
    from stockFunctions import rmsemape
    print("Trend={}".format(td))
    rmsemape(y_test,y_pred)
    print("************")

C:\Users\Hi\anaconda3\envs\aiads\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Trend=n
RMSE-Testset: 0.4192908275180083
maPe-Testset: 1.0885992936092046
************


C:\Users\Hi\anaconda3\envs\aiads\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Trend=t
RMSE-Testset: 0.4818943779896666
maPe-Testset: 1.9849808767983899
************


C:\Users\Hi\anaconda3\envs\aiads\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Trend=c
RMSE-Testset: 0.24513315818010462
maPe-Testset: 1.0512084686467311
************


C:\Users\Hi\anaconda3\envs\aiads\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Trend=ct
RMSE-Testset: 0.292317687247194
maPe-Testset: 1.0736810034354012
************


In [17]:
len(y_test)

18

In [18]:
#i=1
td="c"
model = ARIMA(X_train, order=(0,0,30),trend=td)
model_fit = model.fit()
# make prediction
y_pred= model_fit.predict(len(X_train), len(data1)-1)
#print(y_pred)
from sklearn.metrics import r2_score
mse=mean_squared_error(y_test,y_pred,squared=False)
from stockFunctions import rmsemape
print("Trend={}".format(td))
rmsemape(y_test,y_pred)
print("************")

C:\Users\Hi\anaconda3\envs\aiads\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Trend=c
RMSE-Testset: 0.3017635930761339
maPe-Testset: 1.1546904054910234
************


In [19]:
#pip install Pillow


In [20]:
#from stockFunctions import graph
#graph(y_test,y_pred,"Actual","predicted","Quality data","Time","Char A")

In [21]:
len(data1)

89

In [22]:
y_test

array([[0.6663586 ],
       [0.13216266],
       [0.30591497],
       [0.31146026],
       [0.61829945],
       [0.69316081],
       [0.05452865],
       [0.35397412],
       [0.65249538],
       [0.60351201],
       [0.5961183 ],
       [0.31608133],
       [0.54621072],
       [0.72181146],
       [0.49630314],
       [0.63401109],
       [0.27911275],
       [0.50831793]])

In [23]:
from stockFunctions import conversionSingle
aTestNormTable=conversionSingle(y_test,[column])

In [24]:
aTestNormTable

,TOTAL A
0,0.666359
1,0.132163
2,0.305915
3,0.31146
4,0.618299
5,0.693161
6,0.054529
7,0.353974
8,0.652495
9,0.603512


In [25]:
actual_stock_price_test_ori=Ms.inverse_transform(aTestNormTable)

In [26]:
actual_stock_price_test_ori

array([[7.21],
       [1.43],
       [3.31],
       [3.37],
       [6.69],
       [7.5 ],
       [0.59],
       [3.83],
       [7.06],
       [6.53],
       [6.45],
       [3.42],
       [5.91],
       [7.81],
       [5.37],
       [6.86],
       [3.02],
       [5.5 ]])

In [27]:
actual_stock_price_test_oriA=conversionSingle(actual_stock_price_test_ori,[column])

In [28]:
actual_stock_price_test_oriA

,TOTAL A
0,7.21
1,1.43
2,3.31
3,3.37
4,6.69
5,7.5
6,0.59
7,3.83
8,7.06
9,6.53


In [29]:
from stockFunctions import conversionSingle
aTestNormTable=conversionSingle(y_test,[column])
actual_stock_price_test_ori=Ms.inverse_transform(aTestNormTable)
actual_stock_price_test_oriA=conversionSingle(actual_stock_price_test_ori,[column])

In [30]:
from stockFunctions import conversionSingle
pTestNormTable=conversionSingle(y_pred,[column])
predicted_stock_price_test_ori=Ms.inverse_transform(pTestNormTable)
predicted_stock_price_test_oriP=conversionSingle(predicted_stock_price_test_ori,[column])

In [31]:
#from stockFunctions import graph
#graph(actual_stock_price_test_oriA,predicted_stock_price_test_oriP,"Actual","Predicted","Quality data","Time","Char A")

In [32]:
from stockFunctions import rmsemape
rmsemape(actual_stock_price_test_oriA,predicted_stock_price_test_oriP)

RMSE-Testset: 3.2650820770837683
maPe-Testset: 1.1546904054910234


In [33]:
forecast=model_fit.predict(len(data1), len(data1)+10)

In [34]:
forecast

array([0.59336841, 0.78626721, 0.67505777, 0.64764068, 0.59079236,
       0.43522125, 0.57179425, 0.47411683, 0.59557221, 0.60351815,
       0.56788707])

In [35]:
from stockFunctions import conversionSingle
fTestNormTable=conversionSingle(forecast,["TOTAL A"])
forecast_stock_price_test_ori=Ms.inverse_transform(fTestNormTable)
forecast_stock_price_test_oriF=conversionSingle(forecast_stock_price_test_ori,["TOTAL A fore cast"])

In [36]:
forecast_stock_price_test_oriF

,TOTAL A fore cast
0,6.420246
1,8.507411
2,7.304125
3,7.007472
4,6.392373
5,4.709094
6,6.186814
7,5.129944
8,6.444091
9,6.530066
